In [2]:
import requests, datetime
from pymongo import MongoClient

## Set up
Run the following cells below to set up the backend components

In [2]:
#if you don't have the required packages to run this notebook, run the following cell by uncommenting it
#!pip install -r requirements.txt

### Start MongoDB Instance

Make sure you have docker running in the background

The command below will start two MongoDB instances.
1. The database where our extracted and transformed data will reside will listening on port 27019
2. The database where our sentiment analyzed data will reside will be listening on port 27018

In [5]:
!docker-compose up -d

[+] Running 0/0
 ⠋ news-sentiment-mong Pulling                                             0.1s
 ⠙ news-store-mongo Pulling                                                0.1s
[+] Running 0/2
 ⠙ news-sentiment-mong Pulling                                             0.2s
 ⠹ news-store-mongo Pulling                                                0.2s
[+] Running 0/2
 ⠹ news-sentiment-mong Pulling                                             0.3s
 ⠸ news-store-mongo Pulling                                                0.3s
[+] Running 0/2
 ⠸ news-sentiment-mong Pulling                                             0.4s
 ⠼ news-store-mongo Pulling                                                0.4s
[+] Running 0/2
 ⠼ news-sentiment-mong Pulling                                             0.5s
 ⠴ news-store-mongo Pulling                                                0.5s
[+] Running 0/2
 ⠴ news-sentiment-mong Pulling                                             0.6s
 ⠦ news-store-mongo Pull

### Start the Flask servers

#### Start the extraction server

Run the cell below, and on the terminal, paste the following:
```bash
python3 -m venv venv
source venv/bin/activate
pip install -r requirements.txt
python3 app.py
```

In [4]:
!open -a Terminal ./extract

#### Start the sentiment server

Run the cell below, and on the terminal, paste the following:
```bash
python3 -m venv venv
source venv/bin/activate
pip install -r requirements.txt
python3 app.py
```

In [3]:
!open -a Terminal ./sentiment-analysis

## Pre-init- Cold Start

**Note**: Ignore this section if you have already run this notebook once. Running this section will erase all your data. This section is only meant
if you are running this notebook for the first time, or if you are testing and want to get rid of the data you inserted previously.

With the servers up and running, we will now run the following cells below to ensure that we are initializing an empty database

In [66]:
#lets start with the extractionDB
client = MongoClient('localhost', 27019) ## or MongoClient("localhost:27")
db = client.test_db
db.list_collection_names() #this should be empty

['transformedData', 'test']

In [51]:
end = (datetime.datetime.today()) - datetime.timedelta(days=1)
between = end - datetime.timedelta(days=1)
start = between - datetime.timedelta(days=1)
# start, end = start.strftime('%Y%m%d'), end.strftime('%Y%m%d')

dates = [start.strftime('%m-%d-%Y'), between.strftime('%m-%d-%Y'), end.strftime('%m-%d-%Y')]
dates

['04-24-2023', '04-25-2023', '04-26-2023']

In [68]:
collection = db.transformedData
for item in collection.find({'date': '04-26-2023'}):
    print(item)

In [69]:
collection = db.test
for item in collection.find({'date': '04-26-2023'}):
    print(item)

{'_id': ObjectId('644b3e7bd9d010201fcebe9b'), 'date': '04-26-2023', 'news': 'Let’s say you would like to move but you’re handcuffed to your house by a low mortgage rate. If you move now, you’ll have to pay a much higher interest rate on your next home loan.', 'entities': [], 'search_term': 'Stock Market', 'source': 'nyt', 'symbol': ''}
{'_id': ObjectId('644b3e7bd9d010201fcebe9d'), 'date': '04-26-2023', 'news': 'First Republic Bank is sliding dangerously into a financial maelstrom, one from which an exit appears increasingly difficult.', 'entities': ['First Republic Bank'], 'search_term': 'Stock Market', 'source': 'nyt', 'symbol': ''}
{'_id': ObjectId('644b3e7bd9d010201fcebea0'), 'date': '04-26-2023', 'news': 'British antitrust regulators dealt a major setback on Wednesday to Microsoft’s plans to acquire the video game giant Activision Blizzard for $69 billion, blocking the proposed deal and handing a notable win to government enforcers around the world who want to rein in Big Tech.', '

In [43]:
collection.delete_many({'date': '04-25-2023'})

In [9]:
#drop collection if previous cell wasn't empty
for collection in db.list_collection_names():
    db[collection].drop()

In [10]:
db.list_collection_names() #this should be empty

[]

In [53]:
#lets now repeat for sentimentDB
client = MongoClient('mongodb://localhost:27018') ## or MongoClient("localhost:27")
db = client.test_db
db.list_collection_names() #this should be empty

['test']

In [55]:
collection = db.test
for item in collection.find({'date': '04-26-2023'}):
    print(item)

{'_id': ObjectId('644add2fc9f4c8145cf97ae0'), 'date': '04-26-2023', 'data': {'04-26-2023': {'Stock Market': {'avg_sentiment': [], 'news': [{'Headline': 'Let’s say you would like to move but you’re handcuffed to your house by a low mortgage rate. If you move now, you’ll have to pay a much higher interest rate on your next home loan.', 'sentiment': {'positive': 0.03892926126718521, 'negative': 0.32119178771972656, 'neutral': 0.6398789882659912}, 'source': 'nyt'}, {'Headline': 'First Republic Bank is sliding dangerously into a financial maelstrom, one from which an exit appears increasingly difficult.', 'sentiment': {'positive': 0.00946016050875187, 'negative': 0.9667735695838928, 'neutral': 0.02376626804471016}, 'source': 'nyt'}, {'Headline': 'British antitrust regulators dealt a major setback on Wednesday to Microsoft’s plans to acquire the video game giant Activision Blizzard for $69 billion, blocking the proposed deal and handing a notable win to government enforcers around the world 

In [12]:
#drop collection if previous cell wasn't empty
for collection in db.list_collection_names():
    db[collection].drop()
db.list_collection_names() #this should be empty

[]

## Data Populate

This is a cold start procedure where our application is assumed to go live for the first time

### Extract

In [13]:
#make sure the extraction flask app is listening on port 8001
url = 'http://localhost:8001/test'

response = requests.get(url)

print(response.json())

{'test': 'test'}


In [14]:
endpoints = [
    '/',
    '/nyt',
    '/mediastack-gen',
    '/yfinance'
]
for endpt in endpoints:
    url = f"http://localhost:8001"+endpt
    response = requests.get(url)
    print(response.json())

{'message': 'Successfully pushed data to mongo'}
{'message': 'success'}
{'message': 'Successfully pushed data to mongo'}
{'message': 'Successfully pushed data to mongo'}


### Transform

In [15]:
#make sure the sentiment-analysis flask app is listening on port 8002
url = 'http://localhost:8002/transform'

response = requests.get(url)

print(response.json())

{'message': 'Successfully pushed transformed data to mongo'}


### Load

In [16]:
#call endpoint to run sentiment
#make sure the extraction flask app is listening on port 8001
url = 'http://localhost:8002/sentiment'

response = requests.get(url)

print(response.json())

{'message': 'Data push to sentiment DB- Succeded'}


## The Publicly Accesible Data

In [5]:
client = MongoClient('mongodb://localhost:27018') #connect to sentiment db
db = client.test_db
collection = db.test
cursor = collection.find()
json_result_arr = []
json_result_dict = {}
for row in cursor:
    print(row)
    for key, val in row['data'].items():
        # json_result[key] = val
        print(key)
        for k, v in val.items():
            
            # for k, v in item.items():
            #     print(v)
            for item in v['news']:
                json_result_dict["stock"] = k
                for kk, vv in item.items():
                    
                    json_result_dict[kk] = vv
                # print(item)
            
                json_result_arr.append(json_result_dict)
                json_result_dict = {}
        print(val)
    # 
print(json_result_arr)
# for doc in :
#     for key, val in doc.items():
#         if key == "data":
#             print(val)
    # print(doc)

{'_id': ObjectId('644b41357d7a3aaf77d98690'), 'date': '04-27-2023', 'data': {}}
{'_id': ObjectId('644bbc4d2a6074f914035999'), 'date': '04-24-2023', 'data': {'04-24-2023': {'Stock Market': {'avg_sentiment': [], 'news': [{'symbol': '', 'headline': 'Ivana Trump was best known for her more-is-more approach to life. She and her most famous ex-husband had their differences over the years, but on that front they were united.', 'sentiment': {'positive': 0.09853672236204147, 'negative': 0.06341701000928879, 'neutral': 0.8380463123321533}, 'source': 'nyt'}, {'symbol': '', 'headline': 'Credit Suisse said on Monday that clients withdrew nearly $69 billion from the bank in the first quarter, underscoring the spiraling troubles the embattled Swiss lender faced leading up to a fire sale to its archrival, UBS, in March.', 'sentiment': {'positive': 0.015966622158885002, 'negative': 0.9542132616043091, 'neutral': 0.029820067808032036}, 'source': 'nyt'}]}, 'Apple': {'avg_sentiment': [], 'news': [{'symbol

In [6]:
for item in json_result_arr:
    print(item)

{'stock': 'Stock Market', 'symbol': '', 'headline': 'Ivana Trump was best known for her more-is-more approach to life. She and her most famous ex-husband had their differences over the years, but on that front they were united.', 'sentiment': {'positive': 0.09853672236204147, 'negative': 0.06341701000928879, 'neutral': 0.8380463123321533}, 'source': 'nyt'}
{'stock': 'Stock Market', 'symbol': '', 'headline': 'Credit Suisse said on Monday that clients withdrew nearly $69 billion from the bank in the first quarter, underscoring the spiraling troubles the embattled Swiss lender faced leading up to a fire sale to its archrival, UBS, in March.', 'sentiment': {'positive': 0.015966622158885002, 'negative': 0.9542132616043091, 'neutral': 0.029820067808032036}, 'source': 'nyt'}
{'stock': 'Apple', 'symbol': 'AAPL', 'headline': 'A federal appeals court ruled on Monday that Apple does not have a monopoly in the mobile games market, siding with a lower court’s 2021 ruling that largely gave the tech 

## Insert Historical Data

Up until the previous section, we only inserted data with the assumption of a cold start, aka, we are launching our application for the first time. This section is to populate the whole
database with past news data. More to be added soon.

## Stopping Docker Mongo Instances

In [1]:
!docker-compose down

[+] Running 0/0
 ⠋ Container news-news-sentiment-mong-1  Stopping                          0.1s 
 ⠋ Container news-news-store-mongo-1     Stopping                          0.1s 
[+] Running 0/2
 ⠙ Container news-news-sentiment-mong-1  Stopping                          0.2s 
 ⠙ Container news-news-store-mongo-1     Stopping                          0.2s 
[+] Running 0/2
 ⠹ Container news-news-sentiment-mong-1  Stopping                          0.3s 
 ⠹ Container news-news-store-mongo-1     Stopping                          0.3s 
[+] Running 0/2
 ⠸ Container news-news-sentiment-mong-1  Stopping                          0.4s 
 ⠸ Container news-news-store-mongo-1     Stopping                          0.4s 
[+] Running 0/2
 ⠼ Container news-news-sentiment-mong-1  Stopping                          0.5s 
 ⠼ Container news-news-store-mongo-1     Stopping                          0.5s 
[+] Running 0/2
 ⠴ Container news-news-sentiment-mong-1  Stopping                          0.6s 
 ⠴ Container 